In [1]:
import sqlalchemy as sa
import pandas as pd
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))
import json

In [2]:
# pip install cx_Oracle --upgrade
import cx_Oracle
cx_Oracle.clientversion()

(19, 3, 0, 0, 0)

In [4]:
with open("../aws_creds.json", "r") as f:
    all_creds = json.load(f)

In [4]:
creds = all_creds["oracle-db"]

In [5]:
locals().update(creds)  # Creating local variables from the JSON entries for this database

In [6]:
connxn_string = f"oracle://{username}:{password}@{host}:{port}/{db}"

In [7]:
engine = sa.create_engine(connxn_string)

In [20]:
# engine.url

In [9]:
insp = sa.inspect(engine)

In [10]:
insp.get_schema_names()

['admin',
 'appqossys',
 'ctxsys',
 'dbsnmp',
 'dip',
 'mortgage',
 'outln',
 'rdsadmin',
 'sys',
 'system']

In [11]:
engine.table_names("mortgage")

['auto_insurance_claim',
 'mortgage_property',
 'mortgage_default',
 'mortgage_customer',
 'mortgage_applicant',
 'customer',
 'customer_offer',
 'customer_attrition',
 'customer_activity',
 'auto_insurance_policy']

### Note
The `CREATE SCHEMA` statement does not actually create a schema. Oracle Database automatically creates a schema when you create a user. 
A schema and a user are more or less the same thing in Oracle.
So, you need to use a statement like:

`create user scm identified by scm`


In [12]:
schema_statement = "create user mortgage identified by mortgage"
grant_statement = "GRANT UNLIMITED TABLESPACE TO mortgage"
# alter_statement = "ALTER USER mortgage QUOTA UNLIMITED ON USERS"

In [13]:
with engine.connect() as con:
    con.execute(schema_statement)
    con.execute(grant_statement)

In [14]:
select_statement = "SELECT * FROM all_users"

In [15]:
with engine.connect() as con:
    rs = con.execute(select_statement)
    display(pd.DataFrame(rs))

,0,1,2
0,MORTGAGE,36,2020-10-06 16:55:00
1,ADMIN,35,2020-09-24 21:29:44
2,RDSADMIN,33,2020-08-06 19:02:19
3,CTXSYS,31,2020-08-06 19:02:05
4,APPQOSSYS,30,2020-08-06 19:01:40
5,DBSNMP,29,2020-08-06 19:01:40
6,DIP,14,2020-08-06 18:57:51
7,OUTLN,9,2020-08-06 18:55:53
8,SYSTEM,5,2020-08-06 18:55:52
9,SYS,0,2020-08-06 18:55:52


## Load csvs

In [16]:
csvs = """
Auto Insurance Claim.csv
Auto Insurance Policy.csv
Customer Activity.csv
Customer Attrition.csv
Customer Offers.csv
Customer.csv
Mortgage Applicant.csv
Mortgage Customer.csv
Mortgage Default.csv
Mortgage Property.csv
"""

In [17]:
# from sqlalchemy.types import String

In [18]:
for file in csvs.strip("\n").split("\n"):
    k = file.strip(".csv").lower().replace(" ", "_")
    print(k)
    df = pd.read_csv(f"../cp4d-mortgage-artifacts/{file}", infer_datetime_format=True)
    display(Markdown(f"### {k}"))
    print(df.shape)
    display(df.head())
    df.to_sql(name=k, 
              con=engine, 
              if_exists="replace", 
              schema="mortgage",
#               method="multi", # Oracle oesn't support multi-row insert
#               dtype=type_mapping
             )

auto_insurance_claim


### auto_insurance_claim

(1000, 9)


,CUSTOMER,CLAIM_ID,POLICY_ID,FIRST_NOTICE_OF_LOSS_DATE,RESPONSE,CLAIM_REASON,INCIDENT_SUMMARY,CLAIM_AMOUNT,TOTAL_CLAIM_AMOUNT
0,CH85057,5373215598,PM4788594,2016-08-08,No,Other,NaN,1146.399097,540.141566
1,CH85444,8989129787,MT0378990,2016-10-14,No,Collision,NaN,414.571186,1067.333126
2,CH85444,9636701890,OF0128306,2017-01-02,No,Collision,NaN,1067.725385,611.918994
3,CH85444,9177470640,SX8636429,2017-03-12,No,Scratch/Dent,NaN,807.224559,282.458203
4,CH97539,16599104,AE6086634,2017-05-31,Yes,Hail,NaN,387.364705,701.708239


## Check to make sure it worked

In [19]:
for file in csvs.strip("\n").split("\n"):
    k = file.strip(".csv").lower().replace(" ", "_")
    display(Markdown(f"### {k}"))
    df = pd.read_sql_table(k, con=engine, schema="mortgage")
    print(df.shape)
    table_cols = df.columns.values.tolist()
    all_cols.append(table_cols)
    display(df.head())

### auto_insurance_claim

(1000, 10)


,index,customer,claim_id,policy_id,first_notice_of_loss_date,response,claim_reason,incident_summary,claim_amount,total_claim_amount
0,49,CP32818,9814302766,IT8875116,2016-12-20,No,Hail,None,452.996539,46.104534
1,50,CP32818,6436734208,SV3794628,2017-04-19,No,Collision,None,816.933485,835.200000
2,51,CP85232,2885750792,SN0438007,2017-09-11,No,Scratch/Dent,None,618.596990,438.778537
3,52,CP85232,9840923416,TR3007818,2016-10-17,No,Scratch/Dent,None,237.215549,355.820799
4,53,CP85232,3292720841,XK8058657,2016-03-24,No,Collision,None,4479.546942,302.033971


### auto_insurance_policy

(915, 25)


...

(419, 3)


,index,id,mortgage_default
0,0,101600,YES
1,1,101731,NO
2,2,100548,YES
3,3,101472,NO
4,4,100562,YES


### mortgage_property

(419, 4)


,index,id,saleprice,location
0,0,101472,340000,L100
1,1,100624,190000,L100
2,2,100532,190000,L100
3,3,100403,161000,L100
4,4,100477,192500,L100
